# Créer un classificateur binaire pour détecter les sourires

Dans sa forme la plus élémentaire, la classification d'images consiste à discerner entre deux classes, ou à signaler la présence ou l'absence d'un trait. Dans cette recette, nous allons implémenter un classificateur binaire qui nous dit si une personne sur une photo sourit. Commençons, voulez-vous ?

Vous devrez installer Pillow, ce qui est très facile avec pip

In [ ]:
! pip install Pillow

Nous utiliserons l'ensemble de données SMILEs, situé ici : https://github.com/hromi/SMILEsmileD. Clonez ou téléchargez une version zippée du référentiel à l'emplacement de votre choix. Dans cette recette.

In [6]:
#! wget https://github.com/hromi/SMILEsmileD/archive/refs/heads/master.zip
#! unzip master.zip

Suivez ces étapes pour former un classificateur à partir de zéro sur l'ensemble de données SMILEs

1. Importez tous les packages nécessaires :

In [7]:
import os
import pathlib

import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *

2. Dénissez une fonction pour charger les images et les étiquettes à partir d'une liste de chemins:

In [8]:
def load_images_and_labels(image_paths):
    images = []
    labels = []

    for image_path in image_paths:
        image = load_img(image_path, target_size=(32, 32),
                         color_mode='grayscale')
        image = img_to_array(image)

        label = image_path.split(os.path.sep)[-2]
        label = 'positive' in label
        label = float(label)

        images.append(image)
        labels.append(label)

    return np.array(images), np.array(labels)

Notez que nous chargeons les images en niveaux de gris, et nous encodons les étiquettes en vérifiant si le mot positif est dans le chemin du fichier de l'image

3. Déﬁnissez une fonction pour construire le réseau de neurones. La structure de ce modèle est basée sur LeNet:

In [9]:
def build_network():
    input_layer = Input(shape=(32, 32, 1))
    x = Conv2D(filters=20,
               kernel_size=(5, 5),
               padding='same',
               strides=(1, 1))(input_layer)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
                     strides=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(filters=50,
               kernel_size=(5, 5),
               padding='same',
               strides=(1, 1))(x)
    x = ELU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2),
                     strides=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Flatten()(x)
    x = Dense(units=500)(x)
    x = ELU()(x)
    x = Dropout(0.4)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

Parce qu'il s'agit d'un problème de classification binaire, un seul neurone activé par le sigmoïde suffit dans la couche de sortie

4. Chargez les chemins d'images dans une liste :

In [25]:
files_pattern = (pathlib.Path('/content/') /
                 'SMILEsmileD-master' / 'SMILEs' / '*' / '*' /'*.jpg')
files_pattern = str(files_pattern)
dataset_paths = [*glob.glob(files_pattern)]

5. Utilisez la fonction load_images_and_labels() dénie précédemment pour charger l'ensemble de données en mémoire :

In [28]:
X, y = load_images_and_labels(dataset_paths)


6. Normalisez les images et calculez le nombre d'exemples positifs, négatifs dans l'ensemble de données :

In [29]:
X /= 255.0
total = len(y)
total_positive = np.sum(y)
total_negative = total - total_positive
print(f'Total images: {total}')
print(f'Smile images: {total_positive}')
print(f'Non-smile images: {total_negative}')

Total images: 13165
Smile images: 3690.0
Non-smile images: 9475.0


7. Créez des sous-ensembles d'apprentissage, de test et de validation des données :

In [30]:
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y,
                                     test_size=0.2,
                                     stratify=y,
                                     random_state=999)
(X_train, X_val,
 y_train, y_val) = train_test_split(X_train, y_train,
                                    test_size=0.2,
                                    stratify=y_train,
                                    random_state=999)


8. Instanciez le modèle et compilez-le :

In [31]:
model = build_network()
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

9. Entraînez le modèle. Étant donné que l'ensemble de données est déséquilibré, nous attribuons à chaque classe des poids proportionnels au nombre d'images positives et négatives dans l'ensemble de données :

In [32]:
BATCH_SIZE = 32
EPOCHS = 20
model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          class_weight={
              1.0: total / total_positive,
              0.0: total / total_negative
          })

Epoch 1/20
264/264 [==============================] - 29s 103ms/step - loss: 1.9446 - accuracy: 0.7469 - val_loss: 0.5208 - val_accuracy: 0.8011
Epoch 2/20
264/264 [==============================] - 27s 103ms/step - loss: 1.0119 - accuracy: 0.8218 - val_loss: 0.2957 - val_accuracy: 0.8956
Epoch 3/20
264/264 [==============================] - 27s 103ms/step - loss: 0.8430 - accuracy: 0.8375 - val_loss: 0.2930 - val_accuracy: 0.8899
Epoch 4/20
264/264 [==============================] - 27s 103ms/step - loss: 0.7616 - accuracy: 0.8536 - val_loss: 0.2516 - val_accuracy: 0.8989
Epoch 5/20
264/264 [==============================] - 27s 103ms/step - loss: 0.6666 - accuracy: 0.8675 - val_loss: 0.3665 - val_accuracy: 0.8334
Epoch 6/20
264/264 [==============================] - 27s 103ms/step - loss: 0.6219 - accuracy: 0.8818 - val_loss: 0.3400 - val_accuracy: 0.8614
Epoch 7/20
264/264 [==============================] - 27s 103ms/step - loss: 0.6050 - accuracy: 0.8817 - val_loss: 0.4401 - val_ac

10. Évaluez le modèle sur l'ensemble de test :

In [33]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {test_loss}, accuracy: {test_accuracy}')

83/83 [==============================] - 2s 27ms/step - loss: 0.2938 - accuracy: 0.9028
Loss: 0.29380494356155396, accuracy: 0.9027724862098694


Après 20 époques, le réseau devrait obtenir une précision d'environ 90 % sur l'ensemble de test. Dans la section suivante

Nous venons de former un réseau pour déterminer si une personne sourit ou non sur une photo. Notre première grande tâche a été de prendre les images de l'ensemble de données et de les charger dans un format adapté à notre réseau de neurones. Plus précisément, la fonction load_image_and_labels() est chargée de charger une image en niveaux de gris, de la redimensionner en 32x32x1, puis de la convertir en un tableau numpy. Pour extraire l'étiquette, nous avons regardé le dossier contenant de chaque image : s'il contenait le mot positif, nous avons codé l'étiquette comme 1 ; sinon, nous l'avons encodé en 0 (une astuce que nous avons utilisée ici consistait à convertir un booléen en flottant, comme ceci : float(label)).

Ensuite, nous avons construit le réseau de neurones, qui s'inspire de l'architecture LeNet. Le plus gros point à retenir ici est que, parce qu'il s'agit d'un problème de classification binaire, nous pouvons utiliser un seul neurone activé par le sigmoïde pour discerner entre les deux classes.

Nous avons ensuite pris 20 % des images pour constituer notre ensemble de test, et sur les 80 % restants, nous avons pris 20 % supplémentaires pour créer notre ensemble de validation. Avec ces trois sous-ensembles, nous avons procédé à la formation du réseau sur 20 époques, en utilisant binary_crossentropy comme fonction de perte et rmsprop comme optimiseur.

Pour tenir compte du déséquilibre dans l'ensemble de données (sur les 13 165 images, seules 3 690 contiennent des personnes souriantes, tandis que les 9 475 autres ne le font pas), nous avons passé un dictionnaire class_weight où nous avons attribué un poids inversement proportionnel au nombre d'instances de chaque classe dans l'ensemble de données, forçant efectivement le modèle à accorder plus d'attention à la classe 1.0, qui correspond à smile.

Enfin, nous avons atteint une précision d'environ 90,5% sur l'ensemble de test